In [1]:
import pandas as pd
import sqlite3
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import csv

import seaborn as sns
import matplotlib.pyplot as plt

# Import XGBoost classifier
from xgboost import XGBClassifier

# Import scikit-learn functions
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Import scikit-plot functions
from scikitplot.metrics import plot_roc_curve
from scikitplot.metrics import plot_precision_recall_curve
from scikitplot.metrics import plot_calibration_curve

# Import SciPy function
from scipy.spatial import distance

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [2]:
# Data Directory
data_dir = "../../Data/Big-Data-Cup-2021"
bucketless_data_dir = '../../Data/bdc/data'
tracking_data_dir = "{}/TrackingData".format(data_dir)
db_path = '/Users/keltim01/Documents/databases/'

#connect to database 
conn = sqlite3.connect(db_path + 'bdc_2022.db')
cursor = conn.cursor()

In [3]:
# important numbers for the hockey rink 
ICE_LENGTH = 200
ICE_WIDTH = 85
GOAL_X = ICE_LENGTH - 10
GOAL_Y = ICE_WIDTH / 2
D_ZONE = 75
O_ZONE = ICE_LENGTH - 75

## Using tracking data to find a way to classify teams in creating "danger" on the powerplay by creating space. 

- with tracking data get totally away from shots as basis of structure but take locations of players as structure → also maybe get away from shots as basis at all
- xG everywhere not shots to be able to classify? 
- Combine tracking and event data to look at nearest defender for attackers on the powerplay
- factor in shot selection: xG shoot now, xG Pass etc.
- BART model?

Important questions: 
- Who is good in creating space? 
- Who creates more dangerous situations (more xG)? 
- Who creates danger and space in formation? 
- Who uses good chances (finishing) or selects good shots?  

In [4]:
# Merging Tracking and Play-by-Play data 
df_tracking = pd.read_sql('SELECT * FROM tracking', conn)

In [5]:
df_pbp_pp = pd.read_sql('SELECT * FROM play_by_play WHERE frame_id_1 IS NOT NULL AND strength_state > 0', conn) 

In [6]:
df_pbp_pp.head()

,level_0,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,non_shot_expected_goals,expected_goals
0,441,441,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,375,5 on 4,0,0,Brianne Jenner,Play,1,185,7,Direct,Marie-Philip Poulin,139.0,5.0,None,0,0,344.0,370.0,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,3,0,17,-1,0,2775.0,3,6,5,4,1,35.850384,0.140845,Play,3.0,Indirect,15.0,194.0,66.0,2778.0,23.837995,-0.170213,-3.0,0.311058,12.012389,Puck Recovery,4.0,None,-1.0,165.0,82.0,2780.0,46.746658,0.632911,-2.0,-0.803124,-22.908663,0.189897,NaN
1,442,442,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,373,5 on 4,0,0,Marie-Philip Poulin,Play,1,161,3,Direct,Rebecca Johnston,143.0,19.0,None,0,0,420.0,443.0,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,3,0,60,-1,0,2773.0,3,6,5,4,1,49.002551,0.734177,Play,3.0,Direct,6.0,185.0,7.0,2775.0,35.850384,0.140845,-2.0,0.593332,13.152167,Play,3.0,Indirect,15.0,194.0,66.0,2778.0,23.837995,-0.170213,-3.0,0.311058,12.012389,0.189897,NaN
2,443,443,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,366,5 on 4,0,0,Rebecca Johnston,Play,1,176,3,Direct,Marie-Philip Poulin,130.0,17.0,None,0,0,605.0,639.0,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,3,0,81,-1,0,2766.0,3,6,5,4,1,41.907637,0.354430,Play,3.0,Direct,6.0,161.0,3.0,2773.0,49.002551,0.734177,-7.0,-0.379747,-7.094914,Play,3.0,Direct,6.0,185.0,7.0,2775.0,35.850384,0.140845,-2.0,0.593332,13.152167,0.189897,NaN
3,444,444,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,365,5 on 4,0,0,Marie-Philip Poulin,Play,1,130,17,Direct,Erin Ambrose,133.0,55.0,None,0,0,650.0,670.0,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,3,0,60,-1,0,2765.0,3,6,5,4,1,65.193941,2.352941,Play,3.0,Direct,6.0,176.0,3.0,2766.0,41.907637,0.354430,-1.0,1.998511,23.286305,Play,3.0,Direct,6.0,161.0,3.0,2773.0,49.002551,0.734177,-7.0,-0.379747,-7.094914,0.189897,NaN
4,445,445,8/2/2022,2021,Olympic (Women) - Canada,Olympic (Women) - United States,away,1,363,5 on 4,0,0,Erin Ambrose,Play,1,139,55,Direct,Brianne Jenner,176.0,42.0,None,0,0,701.0,718.0,Olympic (Women) - United States,Olympic (Women) - Canada,5,0,0,3,0,29,-1,0,2763.0,3,6,5,4,1,52.509523,4.080000,Play,3.0,Direct,6.0,130.0,17.0,2765.0,65.193941,2.352941,-2.0,1.727059,-12.684418,Play,3.0,Direct,6.0,176.0,3.0,2766.0,41.907637,0.354430,-1.0,1.998511,23.286305,0.189897,NaN


In [7]:
df_tracking.head()

,index,frame_id,period,track_id,team_id,team_name,jersey_number,x_coord,y_coord,video_shot,game_seconds,venue,player,position,game_id
0,0,96,1,2,Light,ROC,100,176.651307,52.308133,1,2590,away,None,None,4
1,1,96,1,3,Light,ROC,27,158.782222,41.325801,1,2590,away,Veronika Korzhakova,Center,4
2,2,96,1,4,Dark,Finland,10,151.138502,51.883526,1,2590,home,Elisa Holopainen,Left Wing,4
3,3,96,1,5,Light,ROC,100,163.814533,59.253376,1,2590,away,None,None,4
4,4,96,1,8,Dark,Finland,88,165.000000,50.000000,1,2590,home,Ronja Savolainen,Defense,4


In [8]:
test_pbp = df_pbp_pp.loc[df_pbp_pp['frame_id_1'] == 96]

In [9]:
test_tracking = df_tracking[(df_tracking['frame_id'] == 108) & (df_tracking['game_id'] == 5) & (df_tracking['period'] == 1)]

In [10]:
df_tracking.columns

Index(['index', 'frame_id', 'period', 'track_id', 'team_id', 'team_name',
       'jersey_number', 'x_coord', 'y_coord', 'video_shot', 'game_seconds',
       'venue', 'player', 'position', 'game_id'],
      dtype='object')

In [11]:
df_pbp_pp.columns

Index(['level_0', 'index', 'game_date', 'season_year', 'team_name',
       'opp_team_name', 'venue', 'period', 'clock_seconds', 'situation_type',
       'goals_for', 'goals_against', 'player_name', 'event',
       'event_successful', 'x_coord', 'y_coord', 'event_type', 'player_name_2',
       'x_coord_2', 'y_coord_2', 'event_detail_1', 'event_detail_2',
       'event_detail_3', 'frame_id_1', 'frame_id_2', 'home_team', 'away_team',
       'game_id', 'is_shot', 'is_goal', 'event_id', 'team_id', 'player_id',
       'detail_1_code', 'goal_diff', 'game_seconds_remaining', 'event_code',
       'event_type_code', 'skaters_for', 'skaters_against', 'strength_state',
       'distance_to_goal', 'angle_to_goal', 'prev_event', 'prev_event_code',
       'prev_event_type', 'prev_event_type_code', 'prev_event_x_coord',
       'prev_event_y_coord', 'prev_event_game_seconds_remaining',
       'prev_event_distance_to_goal', 'prev_event_angle_to_goal',
       'time_diff_last_event', 'angle_diff_last_event

In [12]:
df_pbp_pp[['frame_id_1','frame_id_2']].info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   frame_id_1  861 non-null    float64
 1   frame_id_2  440 non-null    float64
dtypes: float64(2)
memory usage: 13.6 KB


In [13]:
df_pbp_pp[df_pbp_pp['frame_id_1'] == 867]

,level_0,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,non_shot_expected_goals,expected_goals
718,9787,9787,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,2,566,5 on 4,0,1,Andrea Brandli,Puck Recovery,1,22,56,None,None,NaN,NaN,None,0,0,867.0,NaN,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,0,0,4,3,8,-1,-1,1766.0,4,-1,5,4,1,168.541538,12.444444,Dump In/Out,0.0,None,-1.0,147.0,40.0,1769.0,43.072613,17.200000,-3.0,-4.755556,125.468925,Puck Recovery,4.0,None,-1.0,146.0,44.0,1769.0,44.025561,29.333333,0.0,-12.133333,-0.952948,0.197509,NaN
719,9788,9788,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,2,566,5 on 4,0,1,Andrea Brandli,Play,1,22,56,Direct,Lara Christen,41.0,59.0,None,0,0,867.0,906.0,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,0,0,3,3,8,-1,-1,1766.0,3,6,5,4,1,168.541538,12.444444,Puck Recovery,4.0,None,-1.0,22.0,56.0,1766.0,168.541538,12.444444,0.0,0.000000,0.000000,Dump In/Out,0.0,None,-1.0,147.0,40.0,1769.0,43.072613,17.200000,-3.0,-4.755556,125.468925,0.364651,NaN
750,9949,9949,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,2,200,5 on 4,1,0,Elisa Holopainen,Play,0,34,20,Direct,Nelli Laitinen,36.0,65.0,None,0,0,867.0,NaN,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,0,0,3,1,23,-1,1,1400.0,3,6,5,4,1,157.614244,6.933333,Play,3.0,Direct,6.0,69.0,50.0,1401.0,121.232215,16.133333,-1.0,-9.200000,36.382029,Play,3.0,Direct,6.0,56.0,79.0,1403.0,138.882144,3.671233,-2.0,12.462100,-17.649929,0.328330,NaN


In [14]:
df_pbp_pp.loc[11,'frame_id_1']

1028.0

In [15]:
df_distance = pd.DataFrame(index=df_pbp_pp.index)
dfs_distance = []
for index in df_pbp_pp.index:
    test_pbp = df_pbp_pp.loc[index]
    test_tracking = df_tracking[(df_tracking['frame_id'] == test_pbp['frame_id_1']) & (df_tracking['game_id'] == test_pbp['game_id']) & (df_tracking['period'] == test_pbp['period'])]
    if test_pbp['player_name'] in test_tracking['player'].to_list():
        diff_x1 = test_tracking.loc[(test_tracking['player'] != test_pbp['player_name']) & (test_tracking['venue'] != test_pbp['venue']),'x_coord'] - test_tracking.loc[test_tracking['player'] == test_pbp['player_name'],'x_coord'].values[0]
        diff_x1 = diff_x1.astype(float)
        diff_y1 = abs(test_tracking.loc[(test_tracking['player'] != test_pbp['player_name']) & (test_tracking['venue'] != test_pbp['venue']),'y_coord'] - test_tracking.loc[test_tracking['player'] == test_pbp['player_name'],'y_coord'].values[0])
        pl_distance = np.sqrt(diff_x1**2 + diff_y1**2)
        dfs_distance.append(pl_distance.min())
    else:
        if test_tracking.size > 0: 
            diff_x1 = test_tracking.loc[(test_tracking['player'] != test_pbp['player_name']) & (test_tracking['venue'] != test_pbp['venue']),'x_coord'] - test_pbp['x_coord']
            diff_x1 = diff_x1.astype(float)
            diff_y1 = abs(test_tracking.loc[(test_tracking['player'] != test_pbp['player_name']) & (test_tracking['venue'] != test_pbp['venue']),'y_coord'] - test_pbp['y_coord'])
            pl_distance = np.sqrt(diff_x1**2 + diff_y1**2)
            dfs_distance.append(pl_distance.min())
        else: 
            dfs_distance.append(np.nan)
    


In [16]:
df_pbp_pp['nearest_defender'] = dfs_distance

In [17]:
df_pbp_pp['diff_nearest_defender'] = df_pbp_pp['nearest_defender'] - df_pbp_pp['nearest_defender'].shift(1)

In [18]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] != "Puck Recovery"),'diff_nearest_defender'].describe()

count    242.000000
mean      -2.520629
std       13.875851
min      -79.202106
25%       -7.064740
50%       -0.738430
75%        4.441878
max       37.867777
Name: diff_nearest_defender, dtype: float64

In [20]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] != "Puck Recovery")].sort_values('diff_nearest_defender', ascending=False).head(10)

,level_0,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,non_shot_expected_goals,expected_goals,nearest_defender,diff_nearest_defender
573,8340,8340,14/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Finland,away,3,1137,5 on 4,2,0,Alex Cavallini,Play,1,198,40,Direct,Megan Keller,195.0,39.0,None,0,0,1037.0,1091.0,Olympic (Women) - Finland,Olympic (Women) - United States,2,0,0,3,4,3,-1,2,1137.0,3,6,5,4,1,8.381527,-3.200000,Puck Recovery,4.0,None,-1.0,198.0,40.0,1138.0,8.381527,-3.200000,-1.0,0.000000,0.000000,Dump In/Out,0.0,None,-1.0,65.0,58.0,1141.0,125.957334,8.064516,-3.0,-11.264516,-117.575807,0.263964,NaN,45.523091,37.867777
384,5056,5056,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,627,5 on 4,2,1,Alina Muller,Play,1,195,43,Indirect,Lara Christen,197.0,43.0,None,0,0,5339.0,5380.0,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,0,0,3,3,6,-1,1,627.0,3,15,5,4,1,5.024938,-10.000000,Puck Recovery,4.0,None,-1.0,193.0,11.0,629.0,31.642535,-0.095238,-2.0,-9.904762,-26.617597,Zone Entry,7.0,Carried,2.0,126.0,81.0,633.0,74.687683,1.662338,-4.0,-1.757576,-43.045148,0.249744,NaN,33.703392,29.414377
295,3820,3820,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,1,674,5 on 4,0,0,Andrea Brandli,Play,1,197,43,Indirect,Lara Christen,197.0,43.0,None,0,0,2226.0,2266.0,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,0,0,3,3,8,-1,0,3074.0,3,15,5,4,1,7.017834,-14.000000,Puck Recovery,4.0,None,-1.0,197.0,43.0,3075.0,7.017834,-14.000000,-1.0,0.000000,0.000000,Dump In/Out,0.0,None,-1.0,91.0,76.0,3078.0,104.514353,2.955224,-3.0,-16.955224,-97.496519,0.263964,NaN,91.047926,28.480793
180,2359,2359,8/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Olympic Athletes from Russia,home,1,138,5 on 4,1,0,Minnamari Tuominen,Play,1,130,7,Direct,Nelli Laitinen,144.0,62.0,None,0,0,1625.0,1669.0,Olympic (Women) - Finland,Olympic (Women) - Olympic Athletes from Russia,4,0,0,3,1,66,-1,1,2538.0,3,6,5,4,1,69.715493,1.690141,Play,3.0,Direct,6.0,176.0,12.0,2539.0,33.559648,0.459016,-1.0,1.231124,36.155845,Puck Recovery,4.0,None,-1.0,178.0,10.0,2540.0,34.644624,0.369231,-1.0,0.089786,-1.084976,0.189897,NaN,33.777255,25.825534
580,8353,8353,14/2/2022,2021,Olympic (Women) - United States,Olympic (Women) - Finland,away,3,1107,5 on 4,2,0,Dani Cameranesi,Takeaway,1,159,22,None,None,NaN,NaN,None,0,0,1930.0,NaN,Olympic (Women) - Finland,Olympic (Women) - United States,2,0,0,6,4,21,-1,2,1107.0,6,-1,5,4,1,37.165172,1.512195,Play,3.0,Indirect,15.0,190.0,10.0,1110.0,32.500000,0.000000,-3.0,1.512195,4.665172,Puck Recovery,4.0,None,-1.0,160.0,1.0,1112.0,51.207910,0.722892,-2.0,-0.722892,-18.707910,0.189897,NaN,27.225962,21.300169
829,10436,10436,16/2/2022,2021,Olympic (Women) - Finland,Olympic (Women) - Switzerland,home,3,360,5 on 4,2,0,Elisa Holopainen,Play,1,164,4,Direct,Min

In [21]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] != "Puck Recovery"),['player_name','team_name', 'expected_goals','nearest_defender','diff_nearest_defender']].groupby('player_name').sum().sort_values('diff_nearest_defender', ascending=False).head(10)

,expected_goals,nearest_defender,diff_nearest_defender
player_name,,,
Jenni Hiirikoski,0.061100,130.407287,38.807378
Alex Cavallini,0.000000,117.272172,37.867777
Kelly Pannek,0.010575,45.767107,29.586193
Andrea Brandli,0.000000,107.132817,28.480793
Alina Muller,0.000000,68.692995,22.696592
Dani Cameranesi,0.204388,30.429829,21.300169
Olga Sosina,0.013366,202.813568,17.832818
Anna Savonina,0.000000,60.010206,15.358531
Alex Carpenter,0.030158,63.121836,13.821582


In [22]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] != "Puck Recovery"),['player_name','team_name', 'expected_goals','nearest_defender','diff_nearest_defender']].groupby('player_name').sum().sort_values('diff_nearest_defender', ascending=True).head(10)

,expected_goals,nearest_defender,diff_nearest_defender
player_name,,,
Lara Christen,0.000000,180.911263,-108.170776
Sinja Leemann,0.000000,99.142933,-95.003581
Cayla Barnes,0.016301,84.445780,-68.678921
Anna Shibanova,0.000000,147.859687,-59.700211
Noora Tulus,0.000000,55.926424,-58.884373
Abby Roque,0.000000,21.392594,-57.621528
Elisa Holopainen,0.142919,179.773978,-48.123690
Kendall Coyne Schofield,0.000000,13.778295,-35.565020
Viivi Vainikka,0.329346,43.054672,-34.342182


In [23]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] != "Puck Recovery"),['player_name','team_name', 'expected_goals','nearest_defender','diff_nearest_defender']].groupby('player_name').mean().sort_values('diff_nearest_defender', ascending=False).head(10)

,expected_goals,nearest_defender,diff_nearest_defender
player_name,,,
Alex Cavallini,NaN,58.636086,18.933888
Andrea Brandli,NaN,53.566408,14.240397
Laura Stacey,NaN,21.107548,11.829935
Dani Cameranesi,0.204388,15.214914,10.650085
Kelly Pannek,0.010575,15.255702,9.862064
Jenni Hiirikoski,0.030550,21.734548,6.467896
Alina Muller,NaN,17.173249,5.674148
Emily Clark,NaN,16.706173,5.154447
Anna Savonina,NaN,20.003402,5.119510


In [24]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] != "Puck Recovery") & (df_pbp_pp['player_name'] == 'Andrea Brandli')].sort_values('diff_nearest_defender', ascending=False).head(10)

,level_0,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,non_shot_expected_goals,expected_goals,nearest_defender,diff_nearest_defender
295,3820,3820,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,1,674,5 on 4,0,0,Andrea Brandli,Play,1,197,43,Indirect,Lara Christen,197.0,43.0,None,0,0,2226.0,2266.0,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,0,0,3,3,8,-1,0,3074.0,3,15,5,4,1,7.017834,-14.000000,Puck Recovery,4.0,None,-1.0,197.0,43.0,3075.0,7.017834,-14.000000,-1.0,0.0,0.0,Dump In/Out,0.0,None,-1.0,91.0,76.0,3078.0,104.514353,2.955224,-3.0,-16.955224,-97.496519,0.263964,NaN,91.047926,28.480793
688,9454,9454,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,1,26,5 on 4,0,1,Andrea Brandli,Play,1,160,42,Indirect,Sinja Leemann,149.0,44.0,None,0,0,1263.0,1357.0,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,0,0,3,3,8,-1,-1,2426.0,3,15,5,4,1,30.004166,60.000000,Puck Recovery,4.0,None,-1.0,160.0,42.0,2426.0,30.004166,60.000000,0.0,0.0,0.0,Dump In/Out,0.0,None,-1.0,57.0,81.0,2428.0,138.460283,3.454545,-2.0,56.545455,-108.456117,0.189897,NaN,16.084891,0.000000
303,3835,3835,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,1,640,5 on 4,0,0,Andrea Brandli,Play,1,197,43,Indirect,Sinja Leemann,197.0,43.0,None,0,0,3263.0,3323.0,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,0,0,3,3,8,-1,0,3040.0,3,15,5,4,1,7.017834,-14.000000,Puck Recovery,4.0,None,-1.0,197.0,43.0,3040.0,7.017834,-14.000000,0.0,0.0,0.0,Dump In/Out,0.0,None,-1.0,4.0,38.0,3045.0,186.054428,41.333333,-5.0,-55.333333,-179.036593,0.263964,NaN,NaN,NaN
353,5008,5008,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,717,5 on 4,2,1,Andrea Brandli,Play,1,185,8,Direct,Lara Christen,172.0,22.0,None,0,0,828.0,868.0,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,0,0,3,3,8,-1,1,717.0,3,6,5,4,1,34.860436,0.144928,Puck Recovery,4.0,None,-1.0,185.0,8.0,717.0,34.860436,0.144928,0.0,0.0,0.0,Dump In/Out,0.0,None,-1.0,5.0,71.0,722.0,187.182398,6.491228,-5.0,-6.346301,-152.321962,0.189897,NaN,NaN,NaN


In [25]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] != "Puck Recovery") & (df_pbp_pp['player_name'] == 'Alina Muller')].sort_values('diff_nearest_defender', ascending=False).head(10)

,level_0,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,non_shot_expected_goals,expected_goals,nearest_defender,diff_nearest_defender
384,5056,5056,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,627,5 on 4,2,1,Alina Muller,Play,1,195,43,Indirect,Lara Christen,197.0,43.0,None,0,0,5339.0,5380.0,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,0,0,3,3,6,-1,1,627.0,3,15,5,4,1,5.024938,-10.000000,Puck Recovery,4.0,None,-1.0,193.0,11.0,629.0,31.642535,-0.095238,-2.0,-9.904762,-26.617597,Zone Entry,7.0,Carried,2.0,126.0,81.0,633.0,74.687683,1.662338,-4.0,-1.757576,-43.045148,0.249744,NaN,33.703392,29.414377
706,9483,9483,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,2,1157,5 on 4,0,1,Alina Muller,Takeaway,1,160,83,None,None,NaN,NaN,None,0,0,5775.0,NaN,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,0,0,6,3,6,-1,-1,2357.0,6,-1,5,4,1,50.400893,0.740741,Play,3.0,Indirect,15.0,197.0,35.0,2359.0,10.259142,-0.933333,-2.0,1.674074,40.141751,Puck Recovery,4.0,None,-1.0,198.0,46.0,2359.0,8.732125,-2.285714,0.0,1.352381,1.527018,0.189897,NaN,19.385148,7.822415
812,10162,10162,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,3,970,5 on 4,0,2,Alina Muller,Play,1,173,80,Direct,Lara Stalder,144.0,69.0,None,0,0,3337.0,3382.0,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,0,0,3,3,6,-1,-2,970.0,3,6,5,4,1,41.173414,0.453333,Play,3.0,Direct,6.0,144.0,36.0,972.0,46.456969,7.076923,-2.0,-6.623590,-5.283556,Puck Recovery,4.0,None,-1.0,89.0,41.0,976.0,101.011138,67.333333,-4.0,-60.256410,-54.554169,0.189897,NaN,11.730007,0.970500
707,9772,9772,16/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Finland,away,2,594,5 on 4,0,1,Alina Muller,Faceoff Win,1,169,20,Backhand,Susanna Tapani,NaN,NaN,None,0,0,22.0,NaN,Olympic (Women) - Finland,Olympic (Women) - Switzerland,3,0,0,1,3,6,-1,-1,1794.0,1,0,5,4,1,30.777427,0.933333,Penalty Taken,2.0,Tripping,23.0,190.0,70.0,1794.0,27.500000,0.000000,0.0,0.933333,3.277427,Takeaway,6.0,None,-1.0,157.0,26.0,1794.0,36.895122,2.000000,0.0,-2.000000,-9.395122,0.189897,NaN,3.874448,-15.510700


In [26]:
df_pbp_pp.loc[(df_pbp_pp['x_coord'] >= O_ZONE) & (df_pbp_pp['event'] == "Play") & (df_pbp_pp['x_coord'] >= O_ZONE) ,['player_name','team_name','expected_goals','nearest_defender','diff_nearest_defender']].groupby('player_name').mean().sort_values('diff_nearest_defender', ascending=False).head(10)

,expected_goals,nearest_defender,diff_nearest_defender
player_name,,,
Alex Cavallini,NaN,58.636086,18.933888
Alina Muller,NaN,22.716700,15.192438
Andrea Brandli,NaN,53.566408,14.240397
Kelly Pannek,NaN,14.950353,12.281716
Anna Savonina,NaN,20.003402,5.119510
Jenni Hiirikoski,NaN,20.610433,4.839748
Dominique Ruegg,NaN,13.266123,3.633330
Alex Carpenter,NaN,11.630052,3.128845
Rebecca Johnston,NaN,14.336332,2.837665


In [27]:
df_pbp_pp.loc[(df_pbp_pp['venue'] == 'away') & (df_pbp_pp['team_name']=='Olympic (Women) - Switzerland') & (df_pbp_pp['event'] == 'Shot')]

,level_0,index,game_date,season_year,team_name,opp_team_name,venue,period,clock_seconds,situation_type,goals_for,goals_against,player_name,event,event_successful,x_coord,y_coord,event_type,player_name_2,x_coord_2,y_coord_2,event_detail_1,event_detail_2,event_detail_3,frame_id_1,frame_id_2,home_team,away_team,game_id,is_shot,is_goal,event_id,team_id,player_id,detail_1_code,goal_diff,game_seconds_remaining,event_code,event_type_code,skaters_for,skaters_against,strength_state,distance_to_goal,angle_to_goal,prev_event,prev_event_code,prev_event_type,prev_event_type_code,prev_event_x_coord,prev_event_y_coord,prev_event_game_seconds_remaining,prev_event_distance_to_goal,prev_event_angle_to_goal,time_diff_last_event,angle_diff_last_event,distance_diff_last_event,second_prev_event,second_prev_event_code,second_prev_event_type,second_prev_event_type_code,second_prev_event_x_coord,second_prev_event_y_coord,second_prev_event_game_seconds_remaining,second_prev_event_distance_to_goal,second_prev_event_angle_to_goal,prev_time_diff_last_event,prev_angle_diff_last_event,prev_distance_diff_last_event,non_shot_expected_goals,expected_goals,nearest_defender,diff_nearest_defender
274,3788,3788,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,1,734,5 on 4,0,0,Lara Stalder,Shot,0,65,39,Slapshot,None,NaN,NaN,Missed,1,1,426.0,NaN,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,50,1,0,3134.0,5,19,5,4,1,125.048990,35.714286,Play,3.0,Direct,6.0,71.0,60.0,3135.0,120.279882,6.800000,-1.0,28.914286,4.769108,Play,3.0,Direct,6.0,72.0,48.0,3139.0,118.128108,21.454545,-4.0,-14.654545,2.151774,0.328330,0.082104,16.970051,12.367622
276,3790,3790,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,1,733,5 on 4,0,0,Keely Moy,Shot,0,14,57,Snapshot,None,NaN,NaN,Missed,0,0,477.0,NaN,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,45,1,0,3133.0,5,21,5,4,1,176.596291,12.137931,Puck Recovery,4.0,None,-1.0,14.0,57.0,3133.0,176.596291,12.137931,0.0,0.000000,0.000000,Shot,5.0,Slapshot,19.0,65.0,39.0,3134.0,125.048990,35.714286,-1.0,-23.576355,51.547301,0.328330,0.027452,4.215198,0.000000
345,4996,4996,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,737,5 on 4,2,1,Alina Muller,Shot,0,59,26,Wristshot,None,NaN,NaN,Blocked,1,0,219.0,NaN,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,6,0,1,737.0,5,26,5,4,1,132.035033,7.939394,Play,3.0,Direct,6.0,68.0,42.0,739.0,122.001025,244.000000,-2.0,-236.060606,10.034009,Play,3.0,Direct,6.0,50.0,5.0,741.0,144.935330,3.733333,-2.0,240.266667,-22.934306,0.328330,0.089728,20.563596,-2.322973
349,5002,5002,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,726,5 on 4,2,1,Phoebe Staenz,Shot,0,19,41,Snapshot,None,NaN,NaN,On Net,0,1,561.0,NaN,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,78,2,1,726.0,5,21,5,4,1,171.006579,114.000000,Play,3.0,Direct,6.0,5.0,32.0,726.0,185.297733,17.619048,0.0,96.380952,-14.291155,Play,3.0,Direct,6.0,68.0,83.0,730.0,128.546684,3.012346,-4.0,14.606702,56.751049,0.328330,0.137148,6.479541,-5.793425
351,5004,5004,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,725,5 on 4,2,1,Phoebe Staenz,Shot,0,18,40,Snapshot,None,NaN,NaN,On Net,0,0,596.0,NaN,Olympic (Women) - Olympic Athletes from Russia,Olympic (Women) - Switzerland,0,1,0,5,3,78,2,1,725.0,5,21,5,4,1,172.018168,68.800000,Puck Recovery,4.0,None,-1.0,18.0,40.0,725.0,172.018168,68.800000,0.0,0.000000,0.000000,Shot,5.0,Snapshot,21.0,19.0,41.0,726.0,171.006579,114.000000,-1.0,-45.200000,1.011589,0.328330,0.032940,2.257091,0.000000
368,5036,5036,12/2/2022,2021,Olympic (Women) - Switzerland,Olympic (Women) - Olympic Athletes from Russia,away,3,661,5 on 4,2,1,Rahel Enzler,Shot,0,